In [ ]:
import sys
import os
import pandas as pd
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(parent_dir)
parent_dir

In [ ]:
from analysis_helper import get_analysis_df

(full_df,people_df) = get_analysis_df(parent_dir)

In [ ]:
full_df = full_df[full_df['dt'] > '2010-12-31']
test_4th_innings_stats = full_df[(full_df['innings_number'] == '3') & (full_df['match_type'].str.lower() == 'test')]
test_4th_innings_stats
del full_df

In [ ]:
test_4th_innings_stats['innings_id'] = test_4th_innings_stats['match_id'] + "#" + test_4th_innings_stats['innings_number']
grouping_keys = ['batter_name']
batter_stats = test_4th_innings_stats.groupby(grouping_keys).agg(
    runs=('batter_runs', 'sum'),
    deliveries=('match_id', 'count'),
    innings=('innings_id', 'nunique'),
)
batter_stats = batter_stats.reset_index()
dismissals = test_4th_innings_stats[test_4th_innings_stats['is_wicket'] == 1].groupby(['wicket_player_name']).agg(
    dismissals=('wicket_player_name', 'count')
)
dismissals = dismissals.reset_index()
dismissals['batter_name'] = dismissals['wicket_player_name']

batter_stats = pd.merge(
    batter_stats, 
    dismissals, 
    on=grouping_keys,  # List of columns to join on
    how='left',  # or 'left', 'right', 'outer',
).reset_index()
batter_stats['dismissals'] = batter_stats['dismissals'].replace(0, 1).fillna(1)
del batter_stats['index']
del batter_stats['wicket_player_name']
del dismissals
batter_stats =  batter_stats[batter_stats['innings'] > 20 ]
batter_stats['runs_average'] = batter_stats['runs']/batter_stats['dismissals']
batter_stats['deliveries_average'] = batter_stats['deliveries']/batter_stats['dismissals']
batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['deliveries']
batter_stats['dismissal_percentage'] = 100*batter_stats['dismissals']/batter_stats['innings']
# batter_stats =  batter_stats[batter_stats['runs_average'] > 40 ]

In [ ]:
player_list = []
runs_df = batter_stats.sort_values(['runs'], ascending=False)[:10]
player_list = player_list + runs_df['batter_name'].to_list()
runs_df

In [ ]:
runs_average_df = batter_stats.sort_values(['runs_average'], ascending=False)[:10]
player_list = player_list + runs_average_df['batter_name'].to_list()
runs_average_df

In [ ]:
deliveries_df = batter_stats.sort_values(['deliveries'], ascending=False)[:10]
player_list = player_list + deliveries_df['batter_name'].to_list()
deliveries_df

In [ ]:
deliveries_average_df  = batter_stats.sort_values(['deliveries_average'], ascending=False)[:10]
player_list = player_list + deliveries_average_df['batter_name'].to_list()
deliveries_average_df

In [ ]:
dismissal_percent_df  = batter_stats.sort_values(['dismissal_percentage'], ascending=True)[:10]
player_list = player_list + dismissal_percent_df['batter_name'].to_list()
dismissal_percent_df

In [ ]:
batter_stats.sort_values(['strike_rate'], ascending=False)[:10]


In [ ]:
from collections import Counter
player_freq = Counter(player_list)
sorted_categories = player_freq.most_common()
sorted_categories

In [ ]:
players = ['KS Williamson',
 'DA Warner',
 'D Elgar',
 'AD Mathews',
 'HM Amla',
 'Asad Shafiq',
 'AN Cook',
 'BKG Mendis',
 'JE Root',
 'V Kohli']
len(players)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
import io
import base64
from IPython.display import Image, display

In [ ]:
def save_styled_table(df, filename, title="4th innings stats", 
                     figsize=(12, 8), dpi=300, format='png'):
    """
    Save a beautifully styled DataFrame table to file
    
    Parameters:
    - df: pandas DataFrame
    - filename: output filename (with or without extension)
    - title: table title
    - figsize: figure size (width, height)
    - dpi: image resolution (300 for high quality)
    - format: image format ('png', 'jpg', 'pdf', 'svg')
    """
    
    # Add extension if not provided
    if not any(filename.endswith(ext) for ext in ['.png', '.jpg', '.jpeg', '.pdf', '.svg']):
        filename += f'.{format}'
    
    # Create figure
    fig, ax = plt.subplots(figsize=figsize)
    ax.axis('off')
    
    # Create table
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc='center',
        loc='center',
        bbox=[0, 0, 1, 1]
    )
    
    # Enhanced styling
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 2.5)
    
    # Professional color scheme
    header_color = '#2E86AB'
    row_colors = ['#F8F9FA', '#E9ECEF']
    text_color = '#2C3E50'
    
    # Style header row
    for i in range(len(df.columns)):
        table[(0, i)].set_facecolor(header_color)
        table[(0, i)].set_text_props(weight='bold', color='white', fontsize=12)
        table[(0, i)].set_edgecolor('white')
        table[(0, i)].set_linewidth(2)
        table[(0, i)].set_height(0.08)
    
    # Style data rows
    for i in range(1, len(df) + 1):
        for j in range(len(df.columns)):
            # Alternate row colors
            table[(i, j)].set_facecolor(row_colors[i % 2])
            table[(i, j)].set_text_props(color=text_color, fontsize=10)
            table[(i, j)].set_edgecolor('white')
            table[(i, j)].set_linewidth(1)
            table[(i, j)].set_height(0.06)
    
    # Add title
    plt.title(title, fontsize=18, fontweight='bold', pad=30, color=header_color)
    
    # Save with high quality
    plt.savefig(filename, 
                bbox_inches='tight',
                dpi=dpi,
                facecolor='white',
                edgecolor='none',
                format=format,
                pad_inches=0.2,
                transparent=True)
    
    plt.close()

In [ ]:
import dataframe_image as dfi

for player in players:
    player_df = batter_stats[batter_stats['batter_name'] == player].set_index('batter_name')
    player_df = player_df.round(2)
    player_df.rename(columns=str.upper, inplace=True)
    player_df = player_df.T
    player_df = player_df.reset_index()
    player_df.rename(columns={'index': 'Batter name'}, inplace=True)
    save_styled_table(player_df, f"/mnt/c/Users/91636/Documents/cricketbystats/top_10_4th_innings_batter/images/{player}_4th_innings.png")